## 8.2 LiveData - podstawy

Kontynuujemy aplikację z poprzedniego notatnika - dodamy `LiveData` do aplikacji i rozpoczniemy aktualizację danych na ui.

`LiveData` przechowuje dane, które inne obiekty mogą obserwować i reagować na zmiany. Jest to element tzw. **lifecycle-aware** - gdy podłączamy obserwator do `LiveData`, jest on powiązany z obiektem `LifeCycleOwner` (aktywność/fragment) i wykonuje aktualizacje tylko w stanie aktywnym.

<img src="https://media1.giphy.com/media/tcOAkShGdPpbMpwlMw/giphy.gif" width="150" />

Zmiany rozpocznijmy od dodania `MutableLiveData` do klasy `ScrambleViewModel`, wpierw zmodyfikujmy `currentScrambledWord`.

In [ ]:
private val _currentWordCount = MutableLiveData(0)

Przejdźmy do zmiennej wspomagającej i zwróćmy obiekt `LiveData` - niemutowalny - wystawiamy tylko dane niemutowalne

In [ ]:
val currentWordCount: LiveData<Int>
    get() = _currentWordCount

Następnioe musimy zmodyfikować metodę `getNextWord` w klasie `ScrambleViewModel` - wartość zmiennej `currentScrambledWord` teraz modyfikujemy przez wywołanie metody `setValue`

In [ ]:
_currentScrambledWord.value = String(tempWord)

Kolejne dwie zmienne, które musimy obserwować we fragmencie to `currentWordCount` i `score` - dodajmy `LiveData`

In [ ]:
    private val _score = MutableLiveData(0)
    val score: LiveData<Int>
        get() = _score

    private val _currentWordCount = MutableLiveData(0)
    val currentWordCount: LiveData<Int>
        get() = _currentWordCount

Zmodyfikujmy metody
- `nextWord` - zwróćmy uwagę na wywołanie metody `getValue`

In [ ]:
    fun nextWord(): Boolean {
        return if (currentWordCount.value!! < MAX_NO_OF_WORDS) {
            getNextWord()
            true
        } else false
    }

- `reinitializeData`

In [ ]:
    fun reinitializeData() {
        _score.value = 0
        _currentWordCount.value = 0
        usedWordsList.clear()
        getNextWord()
    }

- w metodzie `getNextWord` musimy wykonać inkrementację, nie możemy zrobić tego wprost

In [ ]:
_currentWordCount.value = currentWordCount.value?.inc()

- podobnie w metodzie `increaseScore`

In [ ]:
    private fun increaseScore() {
        _score.value = _score.value?.plus(SCORE_INCREASE)
    }

Przejdźmy do `ScrambleFragment`, w pierwszej kolejności pozbądźmy się metody `updateNextWordOnScreen` i wszystkich jej wywołań - nie będzie już ona potrzebna. Zmodyfikujmy metodę `onSubmitWord`

In [ ]:
    private fun onSubmitWord() {
        val playerWord = binding.textInputEditText.text.toString()

        if (viewModel.isUserWordCorrect(playerWord)) {
            setErrorTextField(false)
            if (!viewModel.nextWord()) showFinalScoreDialog()
        } else setErrorTextField(true)
    }

Następnie podłączmy obserwator dla trzech `LiveData` w metodzie `onViewCreated`

In [ ]:
        viewModel.currentScrambledWord.observe(viewLifecycleOwner) { newWord ->
            binding.textViewUnscrambledWord.text = newWord}

        viewModel.score.observe(viewLifecycleOwner) {score ->
            binding.score.text = score.toString()}

        viewModel.currentWordCount.observe(viewLifecycleOwner) {wordCount ->
            binding.wordCount.text = getString(
                R.string.word_count, wordCount, MAX_NO_OF_WORDS)}

Metoda `observe` przyjnmuje dwa argumenty
- `LifeCycleOwner` - otrzymujemy przez wywołanie metody `getViewLifecycleOwner`
- lambda (obiekt anonimowy) - parametrem jest tutaj wartość "obserwowana" - zawsze aktualna

Pozostałe metody odpowiednio modysikujemy wykorzystując metody `getValue`

Możemy przetestować aplikację - zwróćmy uwagę że teraz wszystkie elementy layoutu są aktualizowane - ponieważ wykorzystujemy `ViewModel` i `LiveData` nie musimy implementować odpowiednich metod w klasie fragmentu.

<img src="https://media1.giphy.com/media/tcOAkShGdPpbMpwlMw/giphy.gif" width="200" />